In [1]:
import numpy as np
import random
import os
import time
import torch 
from torch import tensor
from torch.autograd import Variable
from copy import deepcopy
from random import shuffle
from tensorboardX import SummaryWriter

In [2]:
from lib.turn_program_into_file import turn_program_into_file
from game.wrappers.game_container import GameContainer

In [3]:
cpus = 4
player_size = 8
game = GameContainer(128, 128)
# Each run taks 2.6 sec per cpu, 38089 = a 24h run.
run_limit = 10
save_cycle = 2
output_dir = "./output/pyramid/"
log_path = './output/logs/'
run = int(time.time())
save_dir = output_dir + str(run) + '/'
writer = SummaryWriter()

In [4]:
# king_hill_ids = list(range(0, cpus))
os.makedirs(os.path.join(output_dir, str(run)))

In [ ]:
# -*- coding: utf-8 -*-
import torch

top = 2

# Use the nn package to define our model and loss function.
p1 = torch.nn.Sequential(
    torch.nn.Linear(top**6, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**5, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**3),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**3, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**5, top**6),
    torch.nn.Sigmoid(),
)


p2 = torch.nn.Sequential(
    torch.nn.Linear(top**6, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**5, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**3),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**3, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**5, top**6),
    torch.nn.Sigmoid(),
)


loss_fn = torch.nn.BCELoss()


def return_result(game_result, number):
    result = 0
    
    for i in game_result:
        if i == number:
            result += 1
        
    return result

def create_player(game_board_pt, player_size):
    game_board_np = deepcopy(game_board_pt.detach().numpy())
    
    for i in range(len(game_board_np)):
        game_board_np[i] = 1.0 if game_board_np[i] > 0.5 else 0.0
    
    game_board_np = np.array(game_board_np, dtype=np.int)
    
    return game_board_np


def run_game(home, away, rounds, player_size):
    home = np.reshape(home, (player_size, player_size))
    away = np.reshape(away, (player_size, player_size))
    
    game.add_players(home, away)
    game.launch(rounds)
    final_board = game.gol.grid()
    final_board = np.array(final_board).flatten()
    
    return return_result(final_board, 1)

def create_loss_board(home, away, rounds, player_size, check_order):
    org_score = run_game(home, away, rounds, player_size)
    loss_board = deepcopy(home)
    tile_board = deepcopy(home)
    top_score = org_score
    
    for i in check_order:
        tile_board[i] = 0.0 if home[i] == 1.0 else 1.0
        tile_result = run_game(tile_board, away, rounds, player_size)
        if tile_result >= top_score:
            loss_board[i] = tile_board[i]
            top_score = tile_result
        else:
            tile_board = deepcopy(loss_board)
    if top_score == org_score:
        shuffle(check_order)
        print('shuffle')
    
    return torch.tensor(loss_board, dtype=torch.float), org_score, top_score, check_order

learning_rate = 0.0002
check_order_p1 = list(range(0, player_size**2))
check_order_p2 = list(range(0, player_size**2))
optimizer_p1 = torch.optim.Adam(p1.parameters(), lr=learning_rate)
optimizer_p2 = torch.optim.Adam(p2.parameters(), lr=learning_rate)

start_p1 = Variable(torch.tensor(torch.rand(player_size * player_size), requires_grad=True))
start_p2 = Variable(torch.tensor(torch.rand(player_size * player_size), requires_grad=True))

for t in range(10000):
    
    p1_board = p1(start_p1)
    p2_board = p2(start_p2)

    with torch.no_grad():
        p1_board_np = create_player(p1_board.clone(), player_size)
        p2_board_np = create_player(p2_board.clone(), player_size)

        p1_loss_board, p1_score, p1_top, check_order_p1 = create_loss_board(p1_board_np, p2_board_np, 100, player_size, check_order_p1)
        p2_loss_board, p2_score, p2_top, check_order_p2 = create_loss_board(p2_board_np, p1_board_np, 100, player_size, check_order_p2)

    loss_p1 = tensor((1 - (p1_score/p1_top))) * loss_fn(p1_board, p1_loss_board)
    loss_p1.backward()

    
    optimizer_p1.step()
    
    loss_p2 = tensor((1 - (p2_score/p2_top))) * loss_fn(p2_board, p2_loss_board)
    loss_p2.backward()
    optimizer_p2.step()
    
    optimizer_p1.zero_grad()
    optimizer_p2.zero_grad()
    
    start_p1 = Variable(torch.tensor(p1_loss_board, requires_grad=True))
    start_p2 = Variable(torch.tensor(p2_loss_board, requires_grad=True))
    
    print(loss_p1, p1_score, p1_top)
    print(loss_p2, p2_score, p2_top)
    
    writer.add_scalars('100/loss', {'loss_p1': loss_p1, 'loss_p2': loss_p2}, t)
    writer.add_scalars('100/top', {'p1_top': p1_top, 'p2_top': p2_top}, t)
    writer.add_scalars('100/score', {'p1_score': p1_score, 'p2_score': p2_score}, t)


    if t % 100 == 0:
        print("Saving players:")
        print(p1_board, p2_board)
        turn_program_into_file(np.reshape(p1_board_np, (8,8)), save_dir + "p1-cycle-" + str(t) + ".rle", "RKH", "EW", "")
        turn_program_into_file(np.reshape(p2_board_np, (8,8)), save_dir + "p2-cycle-" + str(t) + ".rle", "RKH", "EW", "")

tensor(0.6254) 8 174
tensor(0.3542) 62 134
Saving players:
tensor([ 0.5225,  0.4925,  0.4885,  0.5003,  0.4884,  0.4834,  0.4885,
         0.5128,  0.4948,  0.4900,  0.4717,  0.5208,  0.4941,  0.4506,
         0.4369,  0.4917,  0.4511,  0.5212,  0.4622,  0.4482,  0.4953,
         0.5001,  0.4370,  0.5699,  0.5547,  0.4994,  0.4761,  0.5022,
         0.4976,  0.5274,  0.5030,  0.5407,  0.4568,  0.5344,  0.4304,
         0.5077,  0.5103,  0.5177,  0.4847,  0.5269,  0.4521,  0.5207,
         0.4552,  0.4678,  0.4844,  0.5049,  0.5172,  0.5424,  0.5153,
         0.4545,  0.4882,  0.4681,  0.4676,  0.5041,  0.4864,  0.4738,
         0.5334,  0.5299,  0.4744,  0.5054,  0.5216,  0.4829,  0.5156,
         0.4792]) tensor([ 0.4918,  0.4674,  0.5575,  0.5334,  0.4576,  0.5087,  0.5214,
         0.4839,  0.5381,  0.4844,  0.4679,  0.4938,  0.5161,  0.5066,
         0.4864,  0.4936,  0.5505,  0.5187,  0.5249,  0.5282,  0.5356,
         0.4552,  0.4785,  0.4512,  0.4648,  0.5130,  0.4633,  0.4833,


tensor(0.6217) 4 156
tensor(0.3591) 56 131
tensor(0.3470) 63 143
tensor(0.6352) 4 157
tensor(0.4944) 33 201
tensor(0.5294) 25 149
tensor(0.5719) 9 201
tensor(0.6191) 4 188
tensor(0.3716) 75 182
tensor(0.4258) 57 179
tensor(0.5130) 28 166
tensor(0.1595) 98 131
tensor(0.3838) 59 159
tensor(0.5782) 17 188
tensor(0.5868) 9 201
tensor(0.4029) 59 156
tensor(0.4788) 33 156
tensor(1.00000e-02 *
       8.9937) 125 146
tensor(0.5822) 6 144
tensor(0.5644) 16 133
tensor(0.4722) 32 124
tensor(0.1597) 98 131
tensor(0.5920) 5 146
tensor(0.1600) 98 131
tensor(0.5721) 9 143
tensor(0.3294) 65 132
tensor(0.6118) 0 233
tensor(0.1595) 98 131
tensor(0.2401) 85 143
tensor(1.00000e-02 *
       3.6658) 132 140
tensor(0.5960) 6 143
tensor(0.6205) 7 140
tensor(0.6019) 0 113
tensor(0.1587) 98 131
tensor(0.2698) 78 141
tensor(0.5626) 16 136
shuffle
tensor(0.5945) 5 146
tensor(0.) 227 227
shuffle
tensor(0.2028) 83 125
tensor(0.) 141 141
tensor(0.5385) 14 142
tensor(0.3377) 70 154
shuffle
tensor(0.) 138 138
tensor(0

tensor(0.3497) 4 169
tensor(0.5669) 4 181
tensor(0.3168) 4 128
tensor(0.3502) 47 159
tensor(0.4070) 4 133
tensor(0.5443) 4 181
tensor(0.3305) 4 139
tensor(0.4831) 16 141
tensor(0.3912) 0 151
tensor(0.4854) 0 158
tensor(0.1018) 78 138
tensor(0.5014) 4 181
tensor(0.2313) 33 136
tensor(0.4847) 16 141
tensor(0.3014) 3 156
tensor(0.2443) 83 170
tensor(0.3386) 16 151
tensor(0.2400) 59 102
tensor(0.2918) 4 139
tensor(0.4647) 0 158
tensor(0.4513) 4 138
tensor(0.3566) 41 128
tensor(0.3199) 4 139
tensor(0.4352) 4 187
tensor(0.3030) 4 139
tensor(0.1956) 79 127
tensor(0.3855) 9 202
tensor(0.1397) 98 141
tensor(0.3267) 16 151
tensor(0.3940) 16 168
tensor(0.3471) 4 130
tensor(0.1342) 98 141
tensor(0.2133) 15 156
tensor(0.3988) 22 170
tensor(0.2756) 24 142
tensor(0.4677) 12 235
tensor(0.2639) 15 156
tensor(0.2928) 92 196
tensor(0.2126) 33 136
tensor(0.2943) 63 168
tensor(0.2832) 4 169
tensor(0.4667) 4 181
tensor(0.2348) 0 156
tensor(0.4406) 6 115
tensor(0.2997) 4 134
tensor(0.3026) 102 231
tensor(0.2

tensor(0.3230) 4 160
tensor(0.3362) 7 184
tensor(0.3014) 4 160
tensor(0.2657) 4 181
tensor(0.3096) 4 160
tensor(0.2083) 63 159
tensor(0.2657) 16 166
tensor(0.5026) 4 148
tensor(0.2228) 33 170
tensor(0.3112) 51 148
tensor(0.3066) 16 166
tensor(0.3135) 51 148
tensor(0.4914) 0 146
tensor(0.1992) 63 159
tensor(0.2230) 47 135
tensor(0.1158) 82 169
tensor(0.2922) 4 160
tensor(0.4592) 7 184
tensor(0.2941) 4 160
tensor(0.2827) 51 148
tensor(0.2802) 4 160
tensor(0.2105) 63 159
tensor(0.2822) 4 160
tensor(0.1363) 67 169
tensor(0.2307) 24 115
tensor(0.3675) 63 176
tensor(0.2553) 4 160
tensor(0.3341) 3 187
tensor(0.4509) 0 146
tensor(0.2358) 69 148
tensor(0.1883) 33 170
tensor(0.2035) 63 159
tensor(0.1905) 53 149
tensor(0.2055) 63 159
tensor(0.2947) 4 160
tensor(0.3730) 7 184
tensor(0.2759) 4 160
tensor(0.1004) 93 150
tensor(0.2872) 4 160
tensor(0.3261) 63 176
tensor(0.2653) 4 160
tensor(0.3038) 20 143
tensor(0.2694) 4 160
tensor(0.2848) 34 164
tensor(0.2598) 4 160
tensor(0.3733) 63 176
tensor(0.2

tensor(0.1328) 4 160
tensor(0.3286) 53 109
tensor(0.1211) 65 128
tensor(0.1955) 63 163
tensor(0.1947) 24 128
tensor(0.2950) 53 109
tensor(0.1463) 4 160
tensor(0.1084) 93 150
tensor(0.1971) 24 128
tensor(1.00000e-02 *
       7.8348) 98 146
tensor(0.1257) 54 128
tensor(1.00000e-02 *
       3.8243) 129 145
tensor(0.1276) 65 128
tensor(0.2054) 63 163
tensor(0.1524) 4 160
tensor(0.3503) 36 170
tensor(0.1664) 103 171
tensor(0.5294) 11 162
tensor(0.1172) 9 160
tensor(0.1167) 93 150
tensor(0.1195) 74 142
tensor(0.2248) 53 198
tensor(0.1388) 4 160
tensor(0.1958) 114 214
tensor(0.1157) 4 160
tensor(0.2706) 63 176
tensor(0.2675) 53 156
tensor(0.2365) 59 127
tensor(0.1280) 65 128
tensor(0.1929) 63 163
tensor(1.00000e-02 *
       9.8243) 65 128
tensor(0.2202) 63 159
tensor(1.00000e-02 *
       9.7591) 65 128
tensor(0.1854) 63 163
tensor(0.1259) 65 128
tensor(0.5801) 6 109
tensor(0.1659) 24 128
tensor(0.1892) 63 159
tensor(0.1568) 4 160
tensor(0.1743) 63 159
tensor(0.1994) 4 160
tensor(0.2941) 53 19

...

shuffle
tensor(1.00000e-02 *
       3.3814) 86 195
tensor(0.) 139 139
tensor(1.00000e-02 *
       6.8734) 78 226
tensor(0.3633) 3 146
shuffle
tensor(1.00000e-02 *
       5.2485) 86 226
tensor(0.) 139 139
tensor(1.00000e-02 *
       3.6886) 86 195
tensor(0.3107) 20 155
tensor(1.00000e-02 *
       3.3421) 86 195
tensor(0.1318) 72 169
tensor(1.00000e-02 *
       6.0156) 86 226
tensor(1.00000e-02 *
       3.8195) 139 169
shuffle
tensor(1.00000e-02 *
       2.0654) 86 195
tensor(0.) 139 139
tensor(1.00000e-02 *
       5.8931) 86 226
tensor(1.00000e-02 *
       6.3608) 72 139
tensor(1.00000e-02 *
       6.3600) 86 226
tensor(0.4416) 3 150
tensor(1.00000e-02 *
       5.9574) 86 226
tensor(1.00000e-02 *
       5.7790) 139 204
tensor(1.00000e-02 *
       3.7064) 86 195
tensor(1.00000e-02 *
       4.6892) 139 204
tensor(1.00000e-02 *
       3.4377) 86 195
tensor(0.4178) 3 210
tensor(1.00000e-02 *
       5.3630) 86 226
tensor(1.00000e-02 *
       4.9459) 139 204
tensor(1.00000e-02 *
       5.3466

tensor(1.00000e-02 *
       5.8912) 86 151
tensor(0.5037) 3 126
tensor(0.2851) 40 203
tensor(0.4596) 3 126
tensor(1.00000e-02 *
       3.7537) 147 226
tensor(1.00000e-02 *
       3.3721) 139 161
tensor(1.00000e-02 *
       6.0669) 86 151
tensor(1.00000e-02 *
       3.5641) 139 169
tensor(1.00000e-02 *
       2.6920) 86 195
tensor(1.00000e-02 *
       3.6131) 139 169
tensor(1.00000e-02 *
       4.3456) 86 195
tensor(1.00000e-02 *
       3.6196) 139 169
tensor(1.00000e-02 *
       6.4442) 86 151
tensor(1.00000e-02 *
       3.7502) 139 169
tensor(1.00000e-02 *
       3.6276) 86 195
tensor(1.00000e-02 *
       4.0128) 139 169
tensor(1.00000e-02 *
       3.3685) 86 195
tensor(1.00000e-02 *
       3.6117) 139 169
tensor(1.00000e-02 *
       6.0210) 86 151
tensor(1.00000e-02 *
       3.2022) 139 169
shuffle
tensor(0.) 195 195
tensor(1.00000e-02 *
       3.7550) 139 169
tensor(1.00000e-02 *
       3.9046) 147 226
tensor(1.00000e-02 *
       4.1056) 139 169
tensor(0.2094) 86 150
tensor(0.3017) 

tensor(0.1119) 86 235
tensor(1.00000e-02 *
       6.2588) 139 204
tensor(1.00000e-02 *
       6.3208) 147 235
tensor(1.00000e-02 *
       3.8314) 139 204
tensor(0.1340) 86 235
tensor(1.00000e-02 *
       2.1185) 139 155
tensor(0.1259) 86 235
tensor(1.00000e-02 *
       4.8271) 139 204
shuffle
tensor(0.2109) 42 156
tensor(0.) 139 139
tensor(0.1253) 86 235
tensor(0.3571) 3 178
tensor(0.1201) 86 235
tensor(1.00000e-02 *
       2.2152) 139 158
tensor(1.00000e-02 *
       7.5781) 147 235
tensor(0.5828) 3 184
tensor(0.1171) 86 235
tensor(1.00000e-02 *
       1.3957) 139 158
tensor(0.1211) 86 235
tensor(1.00000e-02 *
       1.5216) 139 158
tensor(0.1086) 86 235
tensor(1.00000e-02 *
       1.9325) 139 158
tensor(0.1188) 86 235
tensor(0.3517) 3 178
tensor(0.1188) 86 235
tensor(1.00000e-02 *
       1.5221) 139 158
tensor(0.1258) 86 235
tensor(1.00000e-02 *
       1.3243) 139 158
tensor(0.2597) 40 147
tensor(1.00000e-02 *
       1.5615) 139 158
tensor(0.1444) 86 235
tensor(1.00000e-02 *
       1.

tensor(0.1512) 86 152
tensor(1.00000e-02 *
       3.8390) 139 182
tensor(0.1513) 86 152
tensor(1.00000e-02 *
       3.3648) 139 182
tensor(1.00000e-02 *
       7.3666) 147 235
tensor(1.00000e-02 *
       3.6122) 139 182
shuffle
tensor(1.00000e-02 *
       7.2276) 147 235
tensor(0.) 139 139
tensor(0.1617) 86 152
tensor(1.00000e-02 *
       8.0434) 139 205
tensor(0.1763) 42 196
tensor(1.00000e-02 *
       7.5801) 139 205
tensor(0.1817) 42 218
tensor(1.00000e-02 *
       7.3532) 139 205
tensor(1.00000e-02 *
       6.1183) 147 235
tensor(1.00000e-02 *
       7.6775) 139 205
tensor(0.1621) 86 152
tensor(1.00000e-02 *
       7.3780) 139 205
tensor(0.1638) 42 196
tensor(1.00000e-02 *
       6.8144) 139 205
tensor(0.2683) 17 149
tensor(1.00000e-02 *
       8.0394) 139 205
tensor(1.00000e-02 *
       7.2798) 147 235
tensor(1.00000e-02 *
       7.6449) 139 205
shuffle
tensor(0.3354) 40 182
tensor(0.) 139 139
shuffle
tensor(1.00000e-02 *
       7.2559) 147 235
tensor(0.) 139 139
shuffle
tensor(1.

tensor(0.3498) 40 174
tensor(1.00000e-02 *
       3.0233) 139 182
tensor(0.3357) 40 174
tensor(1.00000e-02 *
       3.2680) 139 182
tensor(0.1602) 42 145
tensor(1.00000e-02 *
       4.4307) 139 182
tensor(0.1591) 42 145
tensor(1.00000e-02 *
       3.3709) 139 182
tensor(0.2546) 42 159
tensor(1.00000e-02 *
       3.8855) 139 182
tensor(0.2996) 40 174
tensor(0.1813) 72 163
tensor(0.2890) 40 174
tensor(1.00000e-02 *
       3.5289) 139 182
tensor(1.00000e-02 *
       8.9122) 147 235
tensor(1.00000e-02 *
       3.5013) 139 182
tensor(0.3151) 40 174
tensor(0.1765) 72 163
tensor(0.3220) 40 174
tensor(1.00000e-02 *
       3.8782) 139 182
shuffle
tensor(0.1534) 42 145
tensor(0.) 139 139
tensor(0.1484) 42 145
tensor(0.1299) 72 151
shuffle
tensor(0.1518) 42 145
tensor(0.) 139 139
tensor(0.3183) 40 174
tensor(1.00000e-02 *
       3.3240) 139 161
tensor(0.1506) 42 145
tensor(0.2015) 3 140
shuffle
tensor(0.3301) 40 174
tensor(0.) 139 139
shuffle
tensor(0.2237) 40 132
tensor(0.) 139 139
tensor(0.2715

In [ ]:
turn_program_into_file(np.reshape(p1_board_np, (8,8)), save_dir + "p1-cycle-" + str(t) + ".rle", "RKH", "EW", "")
turn_program_into_file(np.reshape(p2_board_np, (8,8)), save_dir + "p2-cycle-" + str(t) + ".rle", "RKH", "EW", "")